# Random Packet Generator

This notebook uses [qbreader](https://github.com/qbreader) to get random tossups and bonuses and compile them into a 20-question packet pdf.

Hitting the `> Run all` button above should work. In colab, it takes about 3 minutes to run in full.

In [ ]:
#@title Install qbreader and get latex template (~10s)

print(">>> Installing necessary Python packages...")
!pip install -q qbreader > /dev/null 2>&1
# !pip install -q unidecode > /dev/null 2>&1

print(">>> Getting quizbowl.cls LaTeX class file...")
!wget -O quizbowl.cls https://raw.githubusercontent.com/kevinferg/quizbowl-latex-template/main/quizbowl.cls > /dev/null 2>&1

print(">>> Done.")

In [ ]:
#@title Import packages and define auxiliary functions

from google.colab import files
import subprocess
import re
import unicodedata
from qbreader import Sync as qbr

################################################################################
#   Fix issue where some Category and Subcategory enums don't exist
from qbreader.types import Category, Subcategory

def patch_enum(enum_cls):
    original_new = enum_cls.__new__

    def new(cls, value):
        if value in cls._value2member_map_:
            return original_new(cls, value)
        # fallback to first known member
        first_member = next(iter(cls._value2member_map_))
        return original_new(cls, first_member)

    enum_cls.__new__ = new

patch_enum(Category)
patch_enum(Subcategory)


################################################################################
#   Printing in correct latex format
def html_to_marks(s: str) -> str:
    def bold_underline_repl(match):
        content = match.group(1)
        # wrap everything inside in *...*, even if multiple <u> tags
        content = re.sub(r'<u>(.*?)</u>', r'\1', content, flags=re.DOTALL)
        return f"*{content}*"

    s = re.sub(r'<b>(.*?)</b>', bold_underline_repl, s, flags=re.DOTALL)
    s = re.sub(r'<u>(.*?)</u>', r'~\1~', s, flags=re.DOTALL)
    s = re.sub(r'<i>(.*?)</i>', r'\1', s, flags=re.DOTALL)

    return s


import unicodedata

def to_ascii(s: str) -> str:
    replacements = {
        "“": '"', "”": '"',  # double quotes
        "‘": "'", "’": "'",  # single quotes / apostrophes
        "‛": "'", "„": '"',  # other variants
    }
    for k, v in replacements.items():
        s = s.replace(k, v)
    normalized = unicodedata.normalize('NFKD', s)
    return ''.join(c if ord(c) < 128 else '?' for c in normalized)


def format_tossup(tossup):
    q = r"""

    \begin{tossup}
      \category{%s \hfill %s}
      \question{%s}
      \answer{%s}
    \end{tossup}

    """ %(tossup.category, str(tossup.set),
          tossup.question_sanitized, html_to_marks(tossup.answer))
    return to_ascii(q)

def format_bonus(bonus):
    q = r"""
    \begin{bonus}
      \category{%s \hfill %s}
      \intro{%s}
      \question{%s}
      \answer{%s}
      \question{%s}
      \answer{%s}
      \question{%s}
      \answer{%s}
    \end{bonus}

    """ %(bonus.category, str(bonus.set), bonus.leadin_sanitized,
          bonus.parts_sanitized[0], html_to_marks(bonus.answers[0]),
          bonus.parts_sanitized[1], html_to_marks(bonus.answers[1]),
          bonus.parts_sanitized[2], html_to_marks(bonus.answers[2]))
    return to_ascii(q)


In [ ]:
#@title Download questions using qbreader (~30s)
client = qbr()
tossups = []
bonuses = []
N = 20
print(">>> Getting tossups and bonuses...")
for i in range(N):
    tossup = client.random_tossup()[0]
    tossups.append(tossup)
    print(f"    - Obtained tossup {i+1} of {N}.")
    bonus = client.random_bonus()[0]
    while len(bonus.parts) != 3:
      bonus = client.random_bonus()[0]
    bonuses.append(bonus)
    print(f"    - Obtained bonus  {i+1} of {N}.")

print("\n>>> Done.")


In [ ]:
#@title Create .tex file

#############################################
#   Begin document string
doc_text = r"""
\documentclass{quizbowl}

\tournament{Random Quizbowl Packet}
% \round{Game 1}
\description{Generated using QBReader}

\begin{document}
\maketitle

"""

#############################################
#   Add questions to document string
for (tossup, bonus) in zip(tossups, bonuses):
    doc_text += format_tossup(tossup)
    doc_text += format_bonus(bonus)

#############################################
#   End document string
doc_text += r"""

\end{document}
"""

#############################################
#   Write document string to .tex file
filename = "packet.tex"
with open(filename, "w", encoding="utf-8") as f:
    f.write(doc_text)

print(f">>> Latex source file {filename} created!")


In [ ]:
#@title Install LaTeX (~1min)
print(">>> Updating package lists...")
!apt-get update -qq > /dev/null 2>&1

print(">>> Installing required LaTeX packages...")
!apt-get install -y -qq texlive-latex-base texlive-latex-recommended texlive-latex-extra texlive-lang-english > /dev/null 2>&1

print(">>> Done.")

In [ ]:
#@title Compile and download (~40s)
print(f">>> Compiling {filename}")
subprocess.run(["pdflatex", "-interaction=nonstopmode", filename])
print(f">>> Downloading {filename.replace(".tex", ".pdf")}")
files.download(filename.replace(".tex", ".pdf"))
print(">>> Done.")